In [1]:
import sys
from collections import defaultdict
from itertools import *
import random
import numpy as np
import pdb
from scipy.sparse import *
from scipy.sparse.linalg import svds
import time

from pyspark import SparkContext
sc=SparkContext.getOrCreate()

In [18]:
###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""
def matrix_factorization(R, P, Q, K, steps=10, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        s  = time.time()
        print("Step {} of {}".format(step, steps))
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        print("Execution time for a step: {}".format(time.time()-s))
        if e < 0.001:
            break
    return P, Q.T

###############################################################################

In [3]:
#TRAINSET LOAD
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))

In [5]:
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
R = csr_matrix((data, (rows, cols)))
R.shape

(15375, 37143)

In [13]:
N = R.shape[0]
M = R.shape[1]
R = R.toarray()
K = 3
P = np.random.rand(N,K)
Q = np.random.rand(M,K)

In [14]:
print("Types:")
print("R: ",type(R))
print("P: ",type(P))
print("Q: ",type(Q))

Types:
R:  <class 'numpy.ndarray'>
P:  <class 'numpy.ndarray'>
Q:  <class 'numpy.ndarray'>


In [ ]:
nP , nQ = matrix_factorization(R, P, Q, K)

Step 0 of 10
